In [ ]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
# N = 400
# F = M*E - M为适应度， E 为经济规模
# 斯巴达 M=1 雅典制 M=0.8
# E 斯巴达每期增长0 雅典制 每期增长2%
# 每一期 有千分之一的概率自发突变到另一种制度
# 随机选取u*N个地区 称作竞择力度系数，N是地区的总数。
# p = F_plus / (F + F_plus)
# 按照 p概率进行变异（前提是制度不一样）

def init_df(pct_ones = 0.05):
    # Set the size of the DataFrame
    n_rows = 20
    n_cols = 20
    # Set the percentage of values to be set to 1
    arr_m = np.random.choice([1, 0.8], size=(n_rows, n_cols), p=[1-pct_ones, pct_ones])
    return arr_m, np.ones((n_rows,n_cols))

def get_new_rc(r, c, M):
    arr_choice = np.array([[1,0],[-1,0],[0,1],[0,-1]])
    row = np.random.choice(arr_choice.shape[0])
    v = arr_choice[row].tolist()
    rr, cc = v[0], v[1]
    new_r = r + rr
    if new_r == M.shape[0]:
        new_r = 0
    new_c = c + cc
    if new_c == M.shape[1]:
        new_c = 0
    return new_r, new_c

def iter_df(M, E,u = 0.75):
    # 雅典制地区每年增长2%，斯巴达地区每年增长0%
    E[np.where(M == 0.8)] *= 1.02
    # 每个制度都有概率自发突变到另一个制度
    pct_add = 0.001
    add_arr = np.random.choice([0, 1], size=M.shape, p=[1-pct_add, pct_add])
    idx = np.where(add_arr == 1)

    for r, c in zip(*idx):
        if M[r,c] == 1:
            M[r,c] = 0.8
        else:
            M[r,c] = 1
            
    # u*N个地区，每个地区随机挑选它4个相邻地区的一个，让前者以概率用后者的制度
     # 竞争比率（描述竞争的强度）
    idx_u = np.where(np.random.choice([0, 1], size=M.shape, p=[1-u, u]) == 1)
    M_cp = M.copy()
    for r, c in zip(*idx_u): # 遍历每个u*N地区
        new_r, new_c = get_new_rc(r, c, M)
        F = M_cp[r, c] * E[r, c]
        F_plus = M_cp[new_r, new_c] * E[new_r, new_c]
        p = F_plus / (F + F_plus)
        # 如果两个制度一样，那么什么变化都不发生
        # 如果两个制度不一样，他们会按照这个概率发生制度传染
        if M[r,c] != M_cp[new_r, new_c]:
            if random.random() < p:
                M[r,c] = M_cp[new_r, new_c]
    lenth_increase = np.where(M == 0.8)[0].shape[0]
    print(lenth_increase)
    return M, E, lenth_increase

M , E = init_df()
se1 = {}

for i in tqdm(range(10000)):
    M, E, L = iter_df(M, E, u=0.2)
    se1[i] = L
M , E = init_df()
se2 = {}

for i in tqdm(range(10000)):
    M, E, L = iter_df(M, E, u=0.9)
    se2[i] = L
s1 = pd.Series(se1, name="u=0.2")
s2 = pd.Series(se2, name="u=0.9")
df = pd.concat([s1,s2], axis=1)


In [ ]:
# failed example



# 初始化200*200随机数组
# grid = np.random.rand(200, 200)  

# # 计算概率矩阵 
# up = np.roll(grid, -1, axis=0)  
# down = np.roll(grid, 1, axis=0)
# left = np.roll(grid, -1, axis=1) 
# right = np.roll(grid, 1, axis=1)

# p = up / (up + grid)
# p += down / (down + grid)
# p += left / (left + grid)
# p += right / (right + grid)

# # 用probability矩阵按概率替换中心点
# new_grid = np.random.choice([up, down, left, right], p=p/p.sum(axis=0))

# # 处理边界 
# new_grid[0,:] = grid[0,:]  
# new_grid[-1,:] = grid[-1,:]
# new_grid[:,0] = grid[:,0]
# new_grid[:,-1] = grid[:,-1]

# grid = new_grid